<h1 align="center"> Sensordatenfusion Tutorium 03 </h1>

<h3 align="center"> Erstellen der Prädiktion </h3>

Die Prädiktion hat den Zweck, dass wir eine Zustandsschätzung basierend auf $x_{k-1|k-1}$ erhalten.
Für $x_{k|k-1}$ benötigen wir lediglich noch unsere Evolutions-/Dynamikmatrix $F_{k|k-1}$, die im Zuge des zweiten Tutoriums erstellt wurde .
Für die dazugehörige Kovarianz $P_{k|k-1}$ brauchen wir zusätzlich zu $F$ noch das Prozessrauschen $Q_{k|k-1}$ (in den Folien $D_{k|k-1}$).
Ziel dieses Aufgabenteils ist es also, einen eigenen $Predictor$ zu erstellen, um $x_{k|k-1}$ und $P_{k|k-1}$ zu erhalten. 


Notwendige Imports 


In [ ]:
import scipy as sp
import numpy as np
from stonesoup.base import Property
from stonesoup.base import Predictor
from stonesoup.functions import gauss2sigma, unscented_transform
from stonesoup.types.prediction import GaussianStatePrediction
from stonesoup.types.state import State

In [ ]:
class SdfKalmanPredictor(Predictor)
    def predict_sdf(x, P, F, Q):
        x_pred = F@x
        P_pred = F@P@F.T + Q

        return x_pred, P_pred

    
